In [12]:
# pip install langchain-ollama
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2")



AIMessage(content='Je aime le programmation.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-21T09:48:25.648872Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3822456200, 'load_duration': 1630869200, 'prompt_eval_count': 45, 'prompt_eval_duration': 1123000000, 'eval_count': 7, 'eval_duration': 435000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-aa6f892f-4c10-4aae-bc30-d9e024750cb4-0', usage_metadata={'input_tokens': 45, 'output_tokens': 7, 'total_tokens': 52})

In [13]:
# basic chat invoke / chat interface supports sending message history into input messages
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg
print(ai_msg.content)

'Je aime programmer.\n\n(Note: I used "aime" which is the correct verb conjugation for the present tense of "aimer", which means "to like". If you meant to say something else, please let me know and I\'ll be happy to help!)'

In [19]:
# with chat prompt
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
ai_msg = chain.invoke(
    {
        "input_language": "English",
        "output_language": "Hindi",
        "input": "I love programming.",
    }
)

ai_msg
#print(ai_msg.content)

AIMessage(content='आप प्रोग्रामिंग का बहुत शौकीन हैं। (aap programmining ke baht shaukini hain.)', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-21T10:01:51.6850801Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2992517500, 'load_duration': 47950400, 'prompt_eval_count': 40, 'prompt_eval_duration': 430000000, 'eval_count': 35, 'eval_duration': 2510000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5404132a-d819-4dc2-8c6d-7bb4ca439f3c-0', usage_metadata={'input_tokens': 40, 'output_tokens': 35, 'total_tokens': 75})

In [21]:
# ollama with TOOL Calling

from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(
    model="llama3.2",
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
result.tool_calls

[{'name': 'validate_user',
  'args': {'addresses': ['123 Fake St', '234 Pretend Boulevard'],
   'user_id': 123},
  'id': '74515c72-adf7-418c-973a-8a822afbf939',
  'type': 'tool_call'}]